## Task 1: Import Libraries

In [ ]:
!pip install livelossplot

In [ ]:
from livelossplot.tf_keras import PlotLossesCallback
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from sklearn.metrics import confusion_matrix
from sklearn import metrics

import numpy as np
np.random.seed(42)
import warnings;warnings.simplefilter('ignore')
%matplotlib inline
print('Tensorflow version:', tf.__version__)

Tensorflow version: 2.4.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Task 2: Load and Preprocess Data

In [ ]:
dataset = pd.read_csv('drive/My Drive/COVID/fanndataset.csv')

In [ ]:
dataset

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,107_2b3_Ar_mc_AKGC417L.wav,0.764436,0.385274,230.241583,847.708983,219.029147,0.003799,-308.507111,134.060196,70.642525,35.080532,28.633177,26.920731,21.754404,16.866100,14.011795,11.788765,9.333282,7.660359,7.180143,7.109902,6.527102,5.133047,3.617092,2.865954,2.674544,2.718401,both
1,112_1b1_Lr_sc_Meditron.wav,0.812180,0.261625,54.081644,163.579257,57.616318,0.001328,-388.882416,91.316772,58.754608,29.392786,16.284124,17.134771,22.467335,25.853315,25.901043,22.771620,16.705278,9.312603,3.578771,1.741547,3.774005,7.883593,11.796705,13.675488,12.756596,9.586442,both
2,112_1p1_Pl_sc_Litt3200.wav,0.766854,0.109939,48.312978,100.597763,71.757899,0.002111,-446.186432,79.745964,65.141396,46.387924,28.955418,17.046297,12.205630,13.211909,17.206043,21.156311,23.047333,22.280363,19.368320,15.341600,11.198809,7.629202,4.966582,3.283697,2.509045,2.480290,both
3,114_1b4_Al_mc_AKGC417L.wav,0.610210,0.106756,398.886349,926.092748,716.228809,0.010252,-333.540283,181.542313,28.036003,-5.786147,14.579693,19.413187,20.224482,20.613018,12.931731,11.297018,16.322435,9.553982,0.880633,5.178864,13.179842,8.825580,-2.270075,-0.697363,3.224563,0.383919,both
4,130_1p2_Ll_mc_AKGC417L.wav,0.701345,0.203489,524.380621,1601.745530,295.869209,0.006298,-312.647308,106.552292,48.851780,40.631100,30.075047,28.854115,24.116611,17.958910,15.284420,16.305651,12.668562,14.000759,8.249481,8.443438,7.588146,8.842076,5.023452,6.341448,1.044217,3.484415,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,195_1b1_Pl_sc_Litt3200.wav,0.710651,0.047425,78.285439,118.063732,131.690853,0.004585,-435.218384,55.693665,47.032455,36.134422,26.305277,19.608923,16.408888,15.834595,16.582565,17.520899,17.927320,17.461868,16.044411,13.766324,10.881126,7.815811,5.098623,3.178211,2.243350,2.182152,wheezes
78,198_1b5_Pl_mc_AKGC417L.wav,0.750855,0.454235,224.684066,929.252950,160.524783,0.003340,-313.929901,134.785583,77.755592,51.800743,36.842468,25.040314,17.972136,13.025214,10.431510,9.428408,8.767425,7.833388,7.094052,6.493408,5.976792,5.584500,4.894088,4.211577,3.701142,3.414317,wheezes
79,201_1b1_Ar_sc_Meditron.wav,0.607867,0.013866,408.044254,1221.174075,402.923202,0.009015,-554.969177,146.348465,69.392357,18.522598,13.696607,25.363117,25.090376,13.935756,7.388916,9.959985,13.074433,10.320685,5.598501,4.437710,5.760975,5.583388,3.887356,3.049124,3.380507,3.193319,wheezes
80,206_1b1_Lr_sc_Meditron.wav,0.686101,0.062998,441.237258,1692.640596,146.535696,0.006876,-502.474121,103.542305,67.294434,14.978888,35.339417,34.152344,44.362602,19.539759,18.460901,8.409515,15.524714,6.319329,11.476152,5.462935,10.482950,1.792004,6.100136,0.795582,5.487709,2.259165,wheezes


In [ ]:
db=dataset.copy()

In [ ]:
db_r=db.drop('filename',axis=1)

In [ ]:
db_r

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0.764436,0.385274,230.241583,847.708983,219.029147,0.003799,-308.507111,134.060196,70.642525,35.080532,28.633177,26.920731,21.754404,16.866100,14.011795,11.788765,9.333282,7.660359,7.180143,7.109902,6.527102,5.133047,3.617092,2.865954,2.674544,2.718401,both
1,0.812180,0.261625,54.081644,163.579257,57.616318,0.001328,-388.882416,91.316772,58.754608,29.392786,16.284124,17.134771,22.467335,25.853315,25.901043,22.771620,16.705278,9.312603,3.578771,1.741547,3.774005,7.883593,11.796705,13.675488,12.756596,9.586442,both
2,0.766854,0.109939,48.312978,100.597763,71.757899,0.002111,-446.186432,79.745964,65.141396,46.387924,28.955418,17.046297,12.205630,13.211909,17.206043,21.156311,23.047333,22.280363,19.368320,15.341600,11.198809,7.629202,4.966582,3.283697,2.509045,2.480290,both
3,0.610210,0.106756,398.886349,926.092748,716.228809,0.010252,-333.540283,181.542313,28.036003,-5.786147,14.579693,19.413187,20.224482,20.613018,12.931731,11.297018,16.322435,9.553982,0.880633,5.178864,13.179842,8.825580,-2.270075,-0.697363,3.224563,0.383919,both
4,0.701345,0.203489,524.380621,1601.745530,295.869209,0.006298,-312.647308,106.552292,48.851780,40.631100,30.075047,28.854115,24.116611,17.958910,15.284420,16.305651,12.668562,14.000759,8.249481,8.443438,7.588146,8.842076,5.023452,6.341448,1.044217,3.484415,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,0.710651,0.047425,78.285439,118.063732,131.690853,0.004585,-435.218384,55.693665,47.032455,36.134422,26.305277,19.608923,16.408888,15.834595,16.582565,17.520899,17.927320,17.461868,16.044411,13.766324,10.881126,7.815811,5.098623,3.178211,2.243350,2.182152,wheezes
78,0.750855,0.454235,224.684066,929.252950,160.524783,0.003340,-313.929901,134.785583,77.755592,51.800743,36.842468,25.040314,17.972136,13.025214,10.431510,9.428408,8.767425,7.833388,7.094052,6.493408,5.976792,5.584500,4.894088,4.211577,3.701142,3.414317,wheezes
79,0.607867,0.013866,408.044254,1221.174075,402.923202,0.009015,-554.969177,146.348465,69.392357,18.522598,13.696607,25.363117,25.090376,13.935756,7.388916,9.959985,13.074433,10.320685,5.598501,4.437710,5.760975,5.583388,3.887356,3.049124,3.380507,3.193319,wheezes
80,0.686101,0.062998,441.237258,1692.640596,146.535696,0.006876,-502.474121,103.542305,67.294434,14.978888,35.339417,34.152344,44.362602,19.539759,18.460901,8.409515,15.524714,6.319329,11.476152,5.462935,10.482950,1.792004,6.100136,0.795582,5.487709,2.259165,wheezes


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train,test = train_test_split(db_r, test_size=0.35, random_state=25)

In [ ]:
train = train.reset_index().drop('index',axis=1)

In [ ]:
train

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0.807545,0.038728,127.975122,691.124937,60.028175,0.001375,-581.764465,69.316696,40.597649,26.621887,30.529482,34.307972,28.027939,19.163868,16.791563,19.173996,19.315149,15.245282,10.931542,9.707654,10.660194,10.794199,8.559418,4.948983,2.493612,3.184971,crackles
1,0.752451,0.084153,308.508669,749.480979,522.417491,0.010349,-400.589294,139.787552,37.700733,6.898059,24.065863,27.853914,14.250236,2.143681,1.653594,11.964037,19.231941,12.945238,4.414958,6.357934,10.346388,7.513065,4.043426,3.210498,2.988533,4.310707,healthy_tach
2,0.802575,0.034048,152.486000,804.822942,75.541075,0.001885,-583.074829,95.407684,57.615402,31.388035,28.196554,33.571594,31.646765,22.791107,16.666695,15.631426,14.037863,9.235232,5.834866,7.332276,10.597386,10.617828,7.180680,3.737869,2.680353,3.642111,healthy_tach
3,0.672895,0.236290,480.138811,1355.513883,537.655604,0.007707,-257.610504,154.667953,40.651592,18.123707,20.674263,26.234219,18.626329,12.676470,8.519849,10.128055,14.604397,6.097213,1.273891,7.482585,7.622627,8.425088,1.640831,3.433506,2.925917,3.225507,both
4,0.589300,0.113108,129.177204,171.643464,256.833129,0.006811,-420.497589,128.385330,87.521362,43.323399,13.303924,2.122508,3.481446,8.552647,12.152968,13.184201,12.258185,10.210240,7.958367,6.304224,5.369879,4.581115,3.425288,2.031862,0.901434,0.315802,wheezes
5,0.665153,0.054666,102.144185,135.086251,194.728001,0.005096,-445.989868,85.400375,68.807709,48.430428,30.376947,17.698515,10.359220,6.976554,6.268564,7.292955,9.113059,10.758576,11.518999,11.095322,9.474991,6.840980,3.674320,0.777876,-1.042021,-1.436638,healthy_tach
6,0.579972,0.007268,377.257729,1102.875945,381.315057,0.009158,-626.186890,150.069458,85.632759,35.364605,15.910589,14.684178,16.450880,14.267736,8.560868,3.228925,1.420308,2.405775,3.482875,3.049880,1.262054,-0.895648,-2.135890,-1.856681,-0.645907,0.212141,both
7,0.766854,0.109939,48.312978,100.597763,71.757899,0.002111,-446.186432,79.745964,65.141396,46.387924,28.955418,17.046297,12.205630,13.211909,17.206043,21.156311,23.047333,22.280363,19.368320,15.341600,11.198809,7.629202,4.966582,3.283697,2.509045,2.480290,both
8,0.583052,0.196467,77.367809,178.423299,90.316121,0.003555,-383.195984,125.046707,67.721565,29.676613,24.238520,28.349125,24.264286,18.426462,19.658279,21.461525,15.185410,5.671535,2.517649,5.187778,7.031850,7.445737,10.126855,13.464686,12.161421,6.543431,crackles
9,0.758592,0.368524,253.134191,993.944546,171.528700,0.003507,-319.082733,121.849968,69.631470,38.603012,30.819262,28.417616,23.975420,19.199450,15.927407,13.734909,11.513489,9.857095,8.752335,7.143154,5.082737,3.880864,3.580884,3.212595,2.559915,2.721946,crackles


In [ ]:
test = test.reset_index().drop('index',axis=1)

In [ ]:
test

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0.748240,0.067854,256.713633,729.459862,480.774974,0.005341,-424.439545,134.618546,40.588741,-2.180433,18.084974,42.162281,34.351978,12.638935,3.419680,6.735877,9.323680,7.217322,5.608157,5.571665,3.685215,1.616028,4.242435,8.305383,6.297146,0.845680,healthy_tach
1,0.703660,0.056383,78.707050,121.794657,151.920757,0.003900,-475.704498,73.903481,59.890785,42.949974,28.349552,19.039919,15.174212,15.062948,16.539637,17.784584,17.739532,16.218498,13.727744,11.004416,8.516997,6.341729,4.431839,2.925333,2.112300,2.124392,crackles
2,0.795061,0.035128,405.965217,1361.891367,332.390720,0.003086,-506.764282,81.607742,53.746296,33.435383,25.754423,25.012524,22.011541,15.745337,10.376285,8.657738,8.567073,7.687856,5.696599,4.370874,4.254213,4.779821,5.101821,5.245094,5.118873,4.862780,healthy_tach
3,0.764436,0.385274,230.241583,847.708983,219.029147,0.003799,-308.507111,134.060196,70.642525,35.080532,28.633177,26.920731,21.754404,16.866100,14.011795,11.788765,9.333282,7.660359,7.180143,7.109902,6.527102,5.133047,3.617092,2.865954,2.674544,2.718401,both
4,0.747796,0.237546,265.903413,939.553054,257.136922,0.003710,-337.012909,116.463837,65.482941,36.995613,22.822777,15.594246,11.706391,9.860019,9.531877,10.046036,9.495850,8.632063,8.198485,7.312955,5.329157,3.981688,3.707891,3.893152,4.013822,4.213904,wheezes
5,0.687989,0.093286,72.394390,108.802586,115.721877,0.004012,-433.232727,75.492157,63.085064,47.579201,33.100418,21.965237,14.903544,11.745961,11.768306,13.707855,16.016621,17.481144,17.591732,16.357464,13.981296,10.793495,7.362726,4.433593,2.587756,1.954691,healthy_tach
6,0.819159,0.124444,28.845478,80.433361,45.058061,0.000963,-505.604431,64.153236,57.269493,49.100163,41.262836,34.323360,28.194817,22.832012,18.423115,15.111767,12.799986,11.263599,10.286993,9.705900,9.386689,9.217693,9.068316,8.765997,8.183456,7.333209,crackles
7,0.747595,0.184191,418.169640,1360.627561,296.293877,0.004318,-344.301880,109.853752,43.959488,30.017263,35.571732,27.427805,14.884975,13.179903,18.470304,17.811924,12.043928,9.037848,8.971968,8.389338,7.115365,6.612372,6.803774,6.912106,6.112109,4.537894,both
8,0.623973,0.105226,103.266831,142.755535,196.091715,0.005038,-436.400177,101.900002,76.575188,47.325020,24.073032,10.511778,4.992603,4.363094,6.421144,9.763732,12.909256,14.425171,13.744169,11.420265,8.501143,5.795470,3.701985,2.421970,2.048363,2.460360,wheezes
9,0.783703,0.081824,236.641820,894.202881,254.439026,0.003489,-452.536469,132.747513,69.550285,25.407858,13.682096,13.982791,12.123705,10.368402,12.055839,14.257633,14.353028,12.859717,10.554601,7.372328,4.856861,4.946289,6.491014,6.299244,4.240507,3.289708,healthy_tach


In [ ]:
train_labels = train['label']
train_labels=pd.DataFrame(train_labels)
train_labels

,label
0,crackles
1,healthy_tach
2,healthy_tach
3,both
4,wheezes
5,healthy_tach
6,both
7,both
8,crackles
9,crackles


In [ ]:
val_labels = test['label']
val_labels=pd.DataFrame(val_labels)
val_labels

,label
0,healthy_tach
1,crackles
2,healthy_tach
3,both
4,wheezes
5,healthy_tach
6,crackles
7,both
8,wheezes
9,healthy_tach


In [ ]:
a=[]

In [ ]:
p = val_labels.copy()
a = p['label'].apply(lambda x: [1,0,0,0] if x=='both' else([0,1,0,0] if x=='crackles' else([0,0,1,0] if x=='healthy_tach' else([0,0,0,1]))))

In [ ]:
y_val = pd.DataFrame(list(a))

In [ ]:
b = []

In [ ]:
m = train_labels.copy()
b = m['label'].apply(lambda x: [1,0,0,0] if x=='both' else([0,1,0,0] if x=='crackles' else([0,0,1,0] if x=='healthy_tach' else([0,0,0,1]))))

In [ ]:
y_train = pd.DataFrame(list(b))

In [ ]:
y_val

,0,1,2,3
0,0,0,1,0
1,0,1,0,0
2,0,0,1,0
3,1,0,0,0
4,0,0,0,1
5,0,0,1,0
6,0,1,0,0
7,1,0,0,0
8,0,0,0,1
9,0,0,1,0


In [ ]:
y_train

,0,1,2,3
0,0,1,0,0
1,0,0,1,0
2,0,0,1,0
3,1,0,0,0
4,0,0,0,1
5,0,0,1,0
6,1,0,0,0
7,1,0,0,0
8,0,1,0,0
9,0,1,0,0


In [ ]:
x_train = train.drop('label', axis=1)
x_train

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.807545,0.038728,127.975122,691.124937,60.028175,0.001375,-581.764465,69.316696,40.597649,26.621887,30.529482,34.307972,28.027939,19.163868,16.791563,19.173996,19.315149,15.245282,10.931542,9.707654,10.660194,10.794199,8.559418,4.948983,2.493612,3.184971
1,0.752451,0.084153,308.508669,749.480979,522.417491,0.010349,-400.589294,139.787552,37.700733,6.898059,24.065863,27.853914,14.250236,2.143681,1.653594,11.964037,19.231941,12.945238,4.414958,6.357934,10.346388,7.513065,4.043426,3.210498,2.988533,4.310707
2,0.802575,0.034048,152.486000,804.822942,75.541075,0.001885,-583.074829,95.407684,57.615402,31.388035,28.196554,33.571594,31.646765,22.791107,16.666695,15.631426,14.037863,9.235232,5.834866,7.332276,10.597386,10.617828,7.180680,3.737869,2.680353,3.642111
3,0.672895,0.236290,480.138811,1355.513883,537.655604,0.007707,-257.610504,154.667953,40.651592,18.123707,20.674263,26.234219,18.626329,12.676470,8.519849,10.128055,14.604397,6.097213,1.273891,7.482585,7.622627,8.425088,1.640831,3.433506,2.925917,3.225507
4,0.589300,0.113108,129.177204,171.643464,256.833129,0.006811,-420.497589,128.385330,87.521362,43.323399,13.303924,2.122508,3.481446,8.552647,12.152968,13.184201,12.258185,10.210240,7.958367,6.304224,5.369879,4.581115,3.425288,2.031862,0.901434,0.315802
5,0.665153,0.054666,102.144185,135.086251,194.728001,0.005096,-445.989868,85.400375,68.807709,48.430428,30.376947,17.698515,10.359220,6.976554,6.268564,7.292955,9.113059,10.758576,11.518999,11.095322,9.474991,6.840980,3.674320,0.777876,-1.042021,-1.436638
6,0.579972,0.007268,377.257729,1102.875945,381.315057,0.009158,-626.186890,150.069458,85.632759,35.364605,15.910589,14.684178,16.450880,14.267736,8.560868,3.228925,1.420308,2.405775,3.482875,3.049880,1.262054,-0.895648,-2.135890,-1.856681,-0.645907,0.212141
7,0.766854,0.109939,48.312978,100.597763,71.757899,0.002111,-446.186432,79.745964,65.141396,46.387924,28.955418,17.046297,12.205630,13.211909,17.206043,21.156311,23.047333,22.280363,19.368320,15.341600,11.198809,7.629202,4.966582,3.283697,2.509045,2.480290
8,0.583052,0.196467,77.367809,178.423299,90.316121,0.003555,-383.195984,125.046707,67.721565,29.676613,24.238520,28.349125,24.264286,18.426462,19.658279,21.461525,15.185410,5.671535,2.517649,5.187778,7.031850,7.445737,10.126855,13.464686,12.161421,6.543431
9,0.758592,0.368524,253.134191,993.944546,171.528700,0.003507,-319.082733,121.849968,69.631470,38.603012,30.819262,28.417616,23.975420,19.199450,15.927407,13.734909,11.513489,9.857095,8.752335,7.143154,5.082737,3.880864,3.580884,3.212595,2.559915,2.721946


In [ ]:
x_val = test.drop('label', axis=1)
x_val

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.748240,0.067854,256.713633,729.459862,480.774974,0.005341,-424.439545,134.618546,40.588741,-2.180433,18.084974,42.162281,34.351978,12.638935,3.419680,6.735877,9.323680,7.217322,5.608157,5.571665,3.685215,1.616028,4.242435,8.305383,6.297146,0.845680
1,0.703660,0.056383,78.707050,121.794657,151.920757,0.003900,-475.704498,73.903481,59.890785,42.949974,28.349552,19.039919,15.174212,15.062948,16.539637,17.784584,17.739532,16.218498,13.727744,11.004416,8.516997,6.341729,4.431839,2.925333,2.112300,2.124392
2,0.795061,0.035128,405.965217,1361.891367,332.390720,0.003086,-506.764282,81.607742,53.746296,33.435383,25.754423,25.012524,22.011541,15.745337,10.376285,8.657738,8.567073,7.687856,5.696599,4.370874,4.254213,4.779821,5.101821,5.245094,5.118873,4.862780
3,0.764436,0.385274,230.241583,847.708983,219.029147,0.003799,-308.507111,134.060196,70.642525,35.080532,28.633177,26.920731,21.754404,16.866100,14.011795,11.788765,9.333282,7.660359,7.180143,7.109902,6.527102,5.133047,3.617092,2.865954,2.674544,2.718401
4,0.747796,0.237546,265.903413,939.553054,257.136922,0.003710,-337.012909,116.463837,65.482941,36.995613,22.822777,15.594246,11.706391,9.860019,9.531877,10.046036,9.495850,8.632063,8.198485,7.312955,5.329157,3.981688,3.707891,3.893152,4.013822,4.213904
5,0.687989,0.093286,72.394390,108.802586,115.721877,0.004012,-433.232727,75.492157,63.085064,47.579201,33.100418,21.965237,14.903544,11.745961,11.768306,13.707855,16.016621,17.481144,17.591732,16.357464,13.981296,10.793495,7.362726,4.433593,2.587756,1.954691
6,0.819159,0.124444,28.845478,80.433361,45.058061,0.000963,-505.604431,64.153236,57.269493,49.100163,41.262836,34.323360,28.194817,22.832012,18.423115,15.111767,12.799986,11.263599,10.286993,9.705900,9.386689,9.217693,9.068316,8.765997,8.183456,7.333209
7,0.747595,0.184191,418.169640,1360.627561,296.293877,0.004318,-344.301880,109.853752,43.959488,30.017263,35.571732,27.427805,14.884975,13.179903,18.470304,17.811924,12.043928,9.037848,8.971968,8.389338,7.115365,6.612372,6.803774,6.912106,6.112109,4.537894
8,0.623973,0.105226,103.266831,142.755535,196.091715,0.005038,-436.400177,101.900002,76.575188,47.325020,24.073032,10.511778,4.992603,4.363094,6.421144,9.763732,12.909256,14.425171,13.744169,11.420265,8.501143,5.795470,3.701985,2.421970,2.048363,2.460360
9,0.783703,0.081824,236.641820,894.202881,254.439026,0.003489,-452.536469,132.747513,69.550285,25.407858,13.682096,13.982791,12.123705,10.368402,12.055839,14.257633,14.353028,12.859717,10.554601,7.372328,4.856861,4.946289,6.491014,6.299244,4.240507,3.289708


In [ ]:
x_train = x_train.values.reshape(53,26,1)

In [ ]:
x_val = x_val.values.reshape(29,26,1)

## Task 3: Creating the CNN Model

In [ ]:
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv1D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling1D

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Initialising the CNN
model = Sequential()

# 1st Convolution
model.add(Conv1D(32, 2, padding='same', batch_input_shape=(53, 26, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.5))

# 2nd Convolution layer
model.add(Conv1D(64, 2, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.3))

# Flattening
model.add(Flatten())

# Fully connected layer
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.4))

model.add(Dense(4, activation='softmax'))

## Task 4: Learning Rate Scheduling and Compile the Model

In [ ]:
initial_learning_rate = 0.005
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate = initial_learning_rate,
    decay_steps=5,
    decay_rate=0.96,
    staircase=True
)
optimizer = Adam(learning_rate=lr_schedule)

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (53, 26, 32)              96        
_________________________________________________________________
batch_normalization (BatchNo (53, 26, 32)              128       
_________________________________________________________________
activation (Activation)      (53, 26, 32)              0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (53, 8, 32)               0         
_________________________________________________________________
dropout (Dropout)            (53, 8, 32)               0         
_________________________________________________________________
conv1d_1 (Conv1D)            (53, 8, 64)               4160      
_________________________________________________________________
batch_normalization_1 (Batch (53, 8, 64)               2

## Task 7: Training the Model

In [ ]:
checkpoint = ModelCheckpoint('model_weight.h5', monitor='val_loss',
                            save_weights_only=True, mode='min', verbose=0)
callbacks = [PlotLossesCallback(), checkpoint]

batch_size = 32

history = model.fit(
    x_train, 
    y_train,
    steps_per_epoch = len(x_train) // batch_size,
    validation_data = (x_val,y_val),
    validation_steps = len(x_val) // batch_size,
    epochs=15,
)

Epoch 1/15
1/1 [==============================] - 2s 2s/step - loss: 1.8174 - accuracy: 0.2830
Epoch 2/15
1/1 [==============================] - 0s 50ms/step - loss: 1.7508 - accuracy: 0.2642
Epoch 3/15
1/1 [==============================] - 0s 52ms/step - loss: 1.5808 - accuracy: 0.3585
Epoch 4/15
1/1 [==============================] - 0s 56ms/step - loss: 1.8001 - accuracy: 0.2830
Epoch 5/15
1/1 [==============================] - 0s 50ms/step - loss: 1.6130 - accuracy: 0.3208
Epoch 6/15
1/1 [==============================] - 0s 50ms/step - loss: 1.7493 - accuracy: 0.3208
Epoch 7/15
1/1 [==============================] - 0s 47ms/step - loss: 1.8530 - accuracy: 0.3774
Epoch 8/15
1/1 [==============================] - 0s 215ms/step - loss: 1.6304 - accuracy: 0.3774
Epoch 9/15
1/1 [==============================] - 0s 49ms/step - loss: 1.4024 - accuracy: 0.3396
Epoch 10/15
1/1 [==============================] - 0s 49ms/step - loss: 1.7266 - accuracy: 0.3396
Epoch 11/15
1/1 [=============

In [ ]:
test_loss, test_acc = model.evaluate(x_val,y_val)
print('test_acc: ',test_acc)

1/1 [==============================] - 0s 239ms/step - loss: 1.5596 - accuracy: 0.2264
test_acc:  0.22641509771347046
